In [2]:
# Instalação das bibliotecas necessárias para o funcionamento do projeto
# exa_py: biblioteca para realizar buscas avançadas na internet
# transformers: utilizada para trabalhar com modelos pré-treinados da Hugging Face
# torch: biblioteca de deep learning para manipular tensores e treinar modelos
# accelerate: otimiza a execução de modelos de deep learning
# bitsandbytes: melhora a eficiência do uso de memória em GPUs
# huggingface_hub: permite acessar e gerenciar modelos na Hugging Face
!pip install exa_py transformers torch accelerate bitsandbytes huggingface_hub
!pip install --upgrade gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 96.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [4]:
# Importação de bibliotecas para autenticação de tokens (Google Colab)
from google.colab import userdata
secret_key = userdata.get('TOKEN')  # Recupera a chave secreta para Hugging Face
api_key = userdata.get('EXA_KEY')   # Recupera a chave da API Exa

In [5]:
# Importação das bibliotecas principais para o funcionamento do projeto
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from exa_py import Exa
from huggingface_hub import login
import gradio as gr

# Configuração da API Exa para realizar buscas
exa = Exa(api_key=api_key)  # Inicializa a API Exa com a chave fornecida

# Autenticação no Hugging Face para acessar modelos privados
login(secret_key)  # Faz login na Hugging Face usando o token armazenado

# Carregamento do modelo Llama pré-treinado da Hugging Face
model_id = "meta-llama/Llama-3.2-1B"  # Identificador do modelo na Hugging Face

# Configuração do tokenizador e do modelo do Llama
tokenizer = AutoTokenizer.from_pretrained(model_id)  # Inicializa o tokenizador do modelo
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",        # Define automaticamente se o modelo será executado em GPU ou CPU
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32  # Ajusta o tipo de dados com base no hardware disponível
)

# Configuração do pipeline para geração de texto
text_generator = pipeline(
    "text-generation",        # Define a tarefa de geração de texto
    model=model,              # Utiliza o modelo carregado
    tokenizer=tokenizer,      # Utiliza o tokenizador carregado
    max_new_tokens=500,       # Define o limite de tokens na resposta gerada
    temperature=0.5,          # Controla a aleatoriedade da geração
    top_k=50,                 # Considera as 50 melhores opções por token
    top_p=0.9,                # Usa núcleo de probabilidade acumulada para limitar as opções
    repetition_penalty=1.2,   # Penaliza repetições para melhorar a qualidade do texto
    no_repeat_ngram_size=3    # Evita repetições de frases de 3 palavras
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [6]:
def buscar_no_exa(pergunta):
    """
    Realiza uma busca no Exa e retorna informações formatadas.
    Args:
        pergunta (str): Pergunta do usuário para busca.
    Returns:
        str: Resultados formatados com título, resumo e URL.
    """
    # Faz a busca utilizando a API Exa com configurações específicas
    result = exa.search(
        pergunta,
        type="neural",         # Utiliza busca neural para maior relevância
        use_autoprompt=True,   # Usa autoprompt para melhorar os resultados
        num_results=5,         # Limita a quantidade de resultados retornados
        category="Programming" # Especifica a categoria da busca
    )

    if result and hasattr(result, "results"):
        # Processa e formata os resultados em uma lista legível
        contexto = []
        for idx, item in enumerate(result.results, 1):
            titulo = getattr(item, 'title', 'Sem título')  # Obtém o título
            resumo = getattr(item, 'snippet', 'Sem resumo')  # Obtém o resumo
            url = getattr(item, 'url', 'Sem URL')  # Obtém a URL
            contexto.append(f"{idx}. Título: {titulo}\nResumo: {resumo}\nURL: {url}\n")

        return "\n".join(contexto)  # Retorna os resultados como uma string formatada
    else:
        return "Nenhuma informação relevante encontrada no Exa."  # Retorna mensagem de erro se nenhum dado for encontrado

In [7]:
def perguntar_ao_llama(pergunta):
    """
    Gera uma resposta com o Llama usando apenas seu conhecimento.
    Args:
        pergunta (str): Pergunta feita pelo usuário.
    Returns:
        str: Resposta gerada pelo modelo Llama.
    """
    # Prompt para guiar o modelo Llama a responder de forma clara e objetiva
    prompt_llama = (
        f"Você é um especialista em programação e tecnologia. Responda à seguinte pergunta com uma explicação clara, objetiva e exemplos práticos, sempre que necessário:\n\n"
        f"Pergunta: {pergunta}\n\n"
        f"Responda de forma detalhada."
    )

    # Gera a resposta usando o pipeline de geração de texto
    resposta = text_generator(prompt_llama)
    return resposta[0]["generated_text"]  # Retorna o texto gerado pelo modelo

In [10]:
# Função principal que lida com a lógica
def processar_pergunta(pergunta_usuario):
    # Obter a resposta do Llama com base no próprio conhecimento
    resposta_llama = perguntar_ao_llama(pergunta_usuario)

    # Buscar informações adicionais no Exa
    resultados_exa = buscar_no_exa(pergunta_usuario)

    # Retornar as respostas em formato amigável
    return f"Resposta do Llama: {resposta_llama}", f"Links do Exa: {resultados_exa}"

# Interface do Gradio
interface = gr.Interface(
    fn=processar_pergunta,
    inputs="text",
    outputs=["text", "text"],
    title="Assistente de Perguntas",
    description="Faça uma pergunta e receba respostas baseadas no Llama e Exa."
)

# Executar a aplicação
if __name__ == "__main__":
    interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://9a0d4fe56382ee97ee.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
